### 6 Brute attack

Multiprocessing брутфорс по самым топовым hero и poor пользователям (ROC-AUC Diff 0,240041 для топ 20 hero_users_top и poor_users_top и рельсой 1-20 по первым и последним 5 транзакциям - `5last_brute_2.py`)

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
# from tqdm import tqdm
import random
from model import predict # Функция, позволяет получить предсказание нейронки.
from check_budget import check_budget # функция проверки бюджета. Проверяйте допустимость решения до сабмита
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

bins_path = "nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)

transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются

train_data_path = "../transactions_finetune.csv" # X - train data
train_target_path = "../target_finetune.csv" # y - true target

In [2]:
seed = 26041999
threshold = pd.read_csv(train_target_path).target.mean() # примерный threshold трейна

In [29]:
transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются
t = pd.read_csv(transactions_path)
# t = t[t['user_id'].isin(t.user_id.unique()[::150])]
# split = t.shape[0] // 14
split = 315000
for i in range(4):
    t.iloc[split*i:split*(i+1)].to_csv(f'datasets/sample_submission{i}.csv', index=True)

    # 0 90k
    # 90k 180k
    # 180k 270k
    # ...
    # 1170k 1260k

In [4]:
transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются
t = pd.read_csv(transactions_path)

In [30]:
ts = pd.read_csv('datasets/sample_submission3.csv', index_col=0)
ts.loc[ts.user_id == ts.user_id.unique()[0]].user_id.value_counts()

644130    300
Name: user_id, dtype: int64

In [37]:
import pytorch_lightning as pl
import torch
import pickle
from model import process_for_nn, TransactionsDataset, get_dataloader, device, TransactionsRnn

bins_path = "nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)

transactions_path = "datasets/sample_submission3.csv"    # путь до файла с транзакциями, которые атакуются

train_data_path = "../transactions_finetune.csv" # X - train data
train_target_path = "../target_finetune.csv" # y - true target

threshold = pd.read_csv(train_target_path).target.mean() # примерный threshold трейна


def create_dl(df, bins_path):
    df_transactions = df.dropna().assign(
            hour=lambda x: x.transaction_dttm.dt.hour,
            day=lambda x: x.transaction_dttm.dt.dayofweek,
            month=lambda x: x.transaction_dttm.dt.month,
            number_day=lambda x: x.transaction_dttm.dt.day,
        )

    with open(bins_path, "rb") as f:
        bins = pickle.load(f)
    features = bins.pop("features")
    # features = bins["features"]  ########################## мб можно оптимизировать
    df = process_for_nn(df_transactions, features, bins)
    dataset = TransactionsDataset(df)
    dataloader = get_dataloader(dataset, device, is_validation=True)
    return dataloader


def predictor(dataloader, model):
    preds = []
    users = []
    for data, target in dataloader:
        y_pred = model(data)
        preds.append(y_pred.detach().cpu().numpy())
        users.append(target.detach().cpu().numpy())
    preds = np.concatenate(preds)
    users = np.concatenate(users)
    return pd.DataFrame({"user_id": users, "target": preds[:, 1]})


def check_target_attack(user, target, attacked_target):
    if target.target_true.loc[target.user_id == user].values[0]:
        return target.target.loc[target.user_id == user].values[0] - attacked_target.target.loc[attacked_target.user_id == user].values[0]
    else:
        return attacked_target.target.loc[attacked_target.user_id == user].values[0] - target.target.loc[target.user_id == user].values[0]


pl.seed_everything(26041999)
model = TransactionsRnn()
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
with open(bins_path, "rb") as f:
    bins = pickle.load(f)

random.seed(20230206)
###############################################
# testing df_train last 10
##############################################

# threshold = pd.read_csv("../data/target_finetune.csv").target.mean()  # вероятность, по которой мы считаем таргет

df_transactions = pd.read_csv(transactions_path, index_col=0)
# cutted_df = pd.read_csv(transactions_path)
df_transactions['transaction_dttm'] = pd.to_datetime(df_transactions.transaction_dttm, format='%Y-%m-%d %H:%M:%S')


df_transactions_attacked = df_transactions.copy()

bins_path = "nn_bins.pickle"
model_path = "nn_weights.ckpt"

target = predict(transactions_path, bins_path, model_path)
target['target_true'] = target.target.apply(lambda x: 0 if x <= threshold else 1)
hero_users_top = target.loc[(target.target_true == 1)].sort_values('target')[-20:].user_id.values
poor_users_top = target.loc[(target.target_true == 0)].sort_values('target')[:20].user_id.values

one_idx = target.index[target.target > threshold]  # Эти пользователи похожи на Героя
zero_idx = target.index[target.target <= threshold] # А эти на Неудачника

users = target.user_id.values

one_users = users[one_idx] # defolt - 1
zero_users = users[zero_idx] # norm users - 0

for user in tqdm(users, desc='Total'):
    df_transactions_user = df_transactions.loc[df_transactions.user_id == user].copy()
    user_score = 0
    best_copy_from = 829187
    best_i = 1
    if user in one_users:
        for copy_from in tqdm(poor_users_top, leave=False, desc='poor_users_top'):
            for i in range(1, 20):
                idx_to_first = df_transactions_user.index[df_transactions_user.user_id == user][:5] # айдишники первых 5 транзакций юзера
                idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][:5*i:i] # айдишники первых 5 транзакций воннаби

                idx_to_last = df_transactions_user.index[df_transactions_user.user_id == user][-5:] # айдишники последних 5 транзакций юзера
                idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-5*i::i] # айдишники последних 5 транзакций воннаби

                idx_to = idx_to_first.append(idx_to_last)
                idx_from = idx_from_first.append(idx_from_last)

                sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
                sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
                sign_mask = (sign_to == sign_from)

                df_transactions_user.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
                df_transactions_user.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

                dataloader = create_dl(df_transactions_user, bins_path)

                attacked_target = predictor(dataloader, model)

                current_user_score = check_target_attack(user, target, attacked_target)

                if current_user_score >= user_score:
                    user_score = current_user_score
                    best_copy_from = copy_from
                    best_i = i

    else:
        for copy_from in tqdm(hero_users_top, leave=False, desc='hero_users_top'):
            for i in range(1, 20):
                idx_to_first = df_transactions_user.index[df_transactions_user.user_id == user][:5] # айдишники первых 5 транзакций юзера
                idx_from_first = df_transactions.index[df_transactions.user_id == copy_from][:5*i:i] # айдишники первых 5 транзакций воннаби

                idx_to_last = df_transactions_user.index[df_transactions_user.user_id == user][-5:] # айдишники последних 5 транзакций юзера
                idx_from_last = df_transactions.index[df_transactions.user_id == copy_from][-5*i::i] # айдишники последних 5 транзакций воннаби

                idx_to = idx_to_first.append(idx_to_last)
                idx_from = idx_from_first.append(idx_from_last)

                sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
                sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
                sign_mask = (sign_to == sign_from)

                df_transactions_user.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
                df_transactions_user.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values

                dataloader = create_dl(df_transactions_user, bins_path)

                attacked_target = predictor(dataloader, model)

                current_user_score = check_target_attack(user, target, attacked_target)

                if current_user_score >= user_score:
                    user_score = current_user_score
                    best_copy_from = copy_from
                    best_i = i

    idx_to_first = df_transactions.index[df_transactions.user_id == user][:5]
    idx_from_first = df_transactions.index[df_transactions.user_id == best_copy_from][:5*best_i:best_i]

    idx_to_last = df_transactions.index[df_transactions.user_id == user][-5:]
    idx_from_last = df_transactions.index[df_transactions.user_id == best_copy_from][-5*best_i::best_i]

    idx_to = idx_to_first.append(idx_to_last)
    idx_from = idx_from_first.append(idx_from_last)

    sign_to = np.sign(df_transactions.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(df_transactions.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)

    df_transactions_attacked.loc[idx_to[sign_mask], "mcc_code"] = df_transactions.loc[idx_from[sign_mask], "mcc_code"].values
    df_transactions_attacked.loc[idx_to[sign_mask], "transaction_amt"] = df_transactions.loc[idx_from[sign_mask], "transaction_amt"].values


attacked_path = 'last_brute_split_test.csv'
df_transactions_attacked.to_csv(attacked_path, index=False)

Global seed set to 26041999
Global seed set to 26041999


Total:   0%|          | 0/1050 [00:00<?, ?it/s]

poor_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

poor_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

hero_users_top:   0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [32]:
target

,user_id,target,target_true
0,644130,0.065741,1
1,644135,0.008330,0
2,644567,0.009597,0
3,644723,0.051293,1
4,644801,0.035781,0
...,...,...,...
1045,868183,0.018597,0
1046,868407,0.034842,0
1047,868573,0.021223,0
1048,868694,0.017885,0


In [20]:
ts.user_id.value_counts()

2880    300
3267    300
3418    300
3588    300
Name: user_id, dtype: int64

In [26]:
t.user_id.unique().shape

(4200,)

In [28]:
4200/4*300

315000.0

In [3]:
df_attacked1 = pd.read_csv('datasets/sample_submission0_attacked.csv')
df_attacked2 = pd.read_csv('datasets/sample_submission1_attacked.csv')
df_attacked3 = pd.read_csv('datasets/sample_submission2_attacked.csv')
df_attacked4 = pd.read_csv('datasets/sample_submission3_attacked.csv')

In [4]:
df_attacked = pd.concat([df_attacked1, df_attacked2, df_attacked3, df_attacked4], ignore_index=True)
attacked_path = "brute_atack.csv"
df_attacked.to_csv(attacked_path, index=False)

In [6]:
attac = predict(attacked_path, bins_path, model_path, random_seed=26041999)

Global seed set to 26041999


In [7]:
pred = predict('../sample_submission.csv', bins_path, model_path, random_seed=26041999)
pred['target_true'] = pred.target.apply(lambda x: 0 if x <= threshold else 1)

Global seed set to 26041999


In [9]:
roc_auc_score(pred['target_true'], attac['target'])

0.3809765240904621